PHASE 6
User provides following inputs:
1. State
2. Season
3. Area
4. Annual rainfall
5. Fertilizer usage
6. Pesticide usage

Crop recommendation model predicts Top-3 suitable crops

Yield prediction model predicts expected yield for each recommended crop.

Final output: Crop suggestions with expected yields.

The system is a two-stage machine learning pipeline where a classification model recommends suitable crops and a regression model predicts the expected yield for each recommended crop.

In [3]:
def advisory_system(input_df, crop_model, yield_model,
                    crop_preprocessor, yield_preprocessor, k=3):
    
    # Encode input for crop recommendation
    X_crop_enc = crop_preprocessor.transform(input_df)
    
    # Get top-k crop recommendations
    probs = crop_model.predict_proba(X_crop_enc)
    top_k_idx = np.argsort(probs, axis=1)[:, -k:]
    crops = crop_model.classes_
    
    recommendations = []
    
    for row, idxs in zip(input_df.iterrows(), top_k_idx):
        for idx in idxs[::-1]:
            crop = crops[idx]
            
            # Prepare input for yield prediction
            temp = input_df.copy()
            temp["Crop"] = crop
            
            X_yield_enc = yield_preprocessor.transform(temp)
            predicted_yield = yield_model.predict(X_yield_enc)[0]
            
            recommendations.append((crop, predicted_yield))
    
    return recommendations


In [6]:
import os

os.listdir("../models")


['crop_model.pkl', 'yield_model.pkl']

In [7]:
os.listdir("../data/processed")


['preprocess_crop.pkl', 'preprocess_yield.pkl']

Verifying if the files and preprocessors exist

In [8]:
import pandas as pd
import numpy as np
import joblib

crop_model = joblib.load("../models/crop_model.pkl")
yield_model = joblib.load("../models/yield_model.pkl")
crop_preprocess = joblib.load("../data/processed/preprocess_crop.pkl")
yield_preprocess = joblib.load("../data/processed/preprocess_yield.pkl")


Saving models allows us to separate training from inference, making the system reusable, efficient, and deployable without retraining.

# Crop Advisory System – Final Demo


In [9]:
sample_input = pd.DataFrame([{
    "Crop_Year": 2020,
    "Season": "Kharif",
    "State": "Tamil Nadu",
    "Area": 2.5,
    "Annual_Rainfall": 1200,
    "Fertilizer": 500,
    "Pesticide": 20
}])


In [10]:
results = advisory_system(
    sample_input,
    crop_model,
    yield_model,
    crop_preprocess,
    yield_preprocess,
    k=3
)

pd.DataFrame(results, columns=["Recommended Crop", "Expected Yield"])


,Recommended Crop,Expected Yield
0,Maize,98.940730
1,Jowar,1.559169
2,Rapeseed &Mustard,0.868687


In [11]:
sample_input_2 = pd.DataFrame([{
    "Crop_Year": 2018,
    "Season": "Rabi",
    "State": "Uttar Pradesh",
    "Area": 1.5,
    "Annual_Rainfall": 600,
    "Fertilizer": 300,
    "Pesticide": 10
}])

results_2 = advisory_system(
    sample_input_2,
    crop_model,
    yield_model,
    crop_preprocess,
    yield_preprocess,
    k=3
)

pd.DataFrame(results_2, columns=["Recommended Crop", "Expected Yield"])


,Recommended Crop,Expected Yield
0,Small millets,0.923885
1,Jowar,1.435013
2,Jute,1.776098


CONCLUSION: We validated the system using multiple realistic scenarios with different seasons, regions, and rainfall levels, and observed consistent changes in recommended crops and predicted yields.